In [ ]:
import pandas as pd
import numpy as np
from LX2_masterscan import mz_ml_paths, spectra_2_df


In [ ]:
import pickle
with open('optoins.pkl','rb') as f: options = pickle.load(f)

In [ ]:
print('here')

In [ ]:
mzmls = mz_ml_paths(options)
samples = [p.stem for p in mzmls]

In [ ]:
spectra_dfs = spectra_2_df(options)


In [ ]:
df = spectra_dfs[0] # first file, already teim range and mass range filtered
ms1_peaks = df.loc[df.precursor_id.isna()]

In [ ]:
df.stem.iloc[0]


In [ ]:
def bin_linear_alignment(masses, options):
    # TODO assert masses are ordered
    up_to = None
    for _, mass in masses.iteritems():
        if up_to is None:
            #up_to = mass + tolerance.getTinDA(mass) this is how its done in some places, but reuslt are not identical to below
            up_to = mass + mass / options["MSresolution"].tolerance 
        if mass <= up_to:
            yield up_to
        else:
            up_to = mass + mass / options["MSresolution"].tolerance
            yield up_to


In [ ]:
def ms1_peaks_agg(ms1_peaks, options):
    ms1_peaks.sort_values("mz", inplace=True)

    # binning is done 3 times in lx1, between each fadi filter is performed, we do it at the end intead
    bins1 = list(bin_linear_alignment(ms1_peaks.mz, options))
    bins2 = list(bin_linear_alignment(ms1_peaks.groupby(bins1)["mz"].transform("mean"), options))
    bins3 = list(bin_linear_alignment(ms1_peaks.groupby(bins2)["mz"].transform("mean"),options))

    ms1_peaks['bin_mass'] = bins3 

    # merge mutiple peaks from single scan
    g = ms1_peaks.groupby(["bin_mass", "scan_id"])
    ms1_peaks["scan_cumcount"] = g.cumcount()
    ms1_peaks["merged_mass"] = g["mz"].transform("mean")
    ms1_peaks['merged_inty'] = g['inty'].transform("mean") # NOTE merge is NOT weighted average 
    

    # aggregate results
    agg_df = (
        ms1_peaks.loc[ms1_peaks.scan_cumcount == 0] # use only the first of merged masses
        .assign(
            mass_intensity=lambda x: x.mz * x.merged_inty
        )  # for the weighted average intensity
        .groupby("bin_mass")
        .agg(
            {
                "merged_mass": ["mean", "count"],
                "merged_inty": ["mean", "sum"],
                "mass_intensity": "sum",
            }
        )
        .dropna()
    )
    agg_df.columns = ["_".join(col).strip() for col in agg_df.columns.values]

    # apply fadi filters, in lx1 its done between each bin  process
    fadi_denominator = ms1_peaks.scan_id.unique().shape[0]
    mask_ff = agg_df.merged_mass_count / fadi_denominator >= options['MSfilter']
    agg_df = agg_df[mask_ff]

    # NOTE intensity threshold is do in add_Sample... but lets do it here
    MSthreshold  = options["MSthreshold"]
    mask_inty = agg_df.merged_inty_mean > MSthreshold
    agg_df = agg_df[mask_inty]

    # for reference...weigted_mass shoud not be necesary
    agg_df["weigted_mass"] = agg_df.mass_intensity_sum / agg_df.merged_inty_sum
    # lx1 intensity is wrong because it uses the total number of scans, instead of the numebr of scans with a peak
    agg_df['lx1_bad_inty'] = agg_df.merged_inty_sum / fadi_denominator

    
    agg_df.rename(columns = {'merged_mass_mean':'mz', 'merged_inty_mean':'inty'}, inplace=True)
    return agg_df

In [ ]:
spectra_dfs[0].polarity.iat[0]

In [ ]:
res = {}
for df in spectra_dfs: # first file, already teim range and mass range filtered
    ms1_peaks = df.loc[df.precursor_id.isna()]
    agg_df = ms1_peaks_agg(ms1_peaks, options)
    agg_df['stem'] = df.stem.iloc[0]
    res[df.stem.iloc[0]] = agg_df

In [ ]:
#recalibrate
def recalibrate_mzs(mzs, cals):
    # from lx2_masterscan
    # lx1 takes all that are within tolerance and then uses highest intensity
    if not cals or mzs.empty:
        return mzs
    cal_matchs = [mzs.loc[mzs.sub(cal).abs().idxmin()] for cal in cals]
    
    cal_vals = [cal - cal_match for cal, cal_match in zip(cals, cal_matchs)]
    # prefilter
    if not any((v < 0.1 for v in cal_vals)):
        return mzs
    # find near tolerance
    cutoff = mzs.diff(-1).quantile(0.1)
    is_near = [v < cutoff for v in cal_vals]
    if not any(is_near):
        log.info("no valid calibration masses found")
        return mzs

    cal_matchs = [e for e, v in zip(cal_matchs, is_near) if v]
    cal_vals = [e for e, v in zip(cal_vals, is_near) if v]
    log.debug("recalibration info: {'\n'.join(zip(cal_matchs,cal_vals ))}")

    return mzs + np.interp(mzs, cal_matchs, cal_vals)

In [ ]:
#align ms1 
ms1_agg_peaks = pd.concat(res.values())

In [ ]:
ms1_agg_peaks

In [ ]:
# this one uses delta res for clustering
def bin_mkSurveyLinear(masses, options):
    # TODO assert masses are ordered
    minmass = masses.iloc[0]
    up_to = None
    for _, mass in masses.iteritems():
        if up_to is None:
            #up_to = mass + tolerance.getTinDA(mass) this is how its done in some places, but reuslt are not identical to below
            deltatol = options["MSresolution"].tolerance + (mass - minmass) * options['MSresolutionDelta']
            up_to = mass + (mass / deltatol)
        if mass <= up_to:
            yield up_to
        else:
            deltatol = options["MSresolution"].tolerance + (mass - minmass) * options['MSresolutionDelta']
            up_to = mass + (mass / deltatol)
            yield up_to

In [ ]:
# make the method here

In [ ]:
ms1_agg_peaks.sort_values('mz', inplace=True)

In [ ]:
# binning is done 3 times in lx1, between each fadi filter is performed, we do it at the end intead
bins1 = list(bin_mkSurveyLinear(ms1_agg_peaks.mz, options))
bins2 = list(bin_mkSurveyLinear(ms1_agg_peaks.groupby(bins1)["mz"].transform("mean"), options))
bins3 = list(bin_mkSurveyLinear(ms1_agg_peaks.groupby(bins2)["mz"].transform("mean"),options))

In [ ]:
ms1_agg_peaks['bins'] = bins3

In [ ]:
# check occupation spectracontainer.py masterscan.chekoccupation
# occupation is the % of peak intensities abvove "thrsld: "
threshold_denominator = ms1_agg_peaks.stem.unique().shape[0] # same as len(res)
threshold = options["MSminOccupation"]
bin_peak_count = ms1_agg_peaks.groupby('bins')['inty'].transform('count')
tf_mask = (bin_peak_count / threshold_denominator) >= threshold
ms1_agg_peaks['above_threshold'] = tf_mask

In [ ]:
ms1_agg_peaks['mass'] = ms1_agg_peaks.groupby('bins')['mz'].transform('mean')

In [ ]:
ms1_agg_peaks

In [ ]:
ms1_agg_peaks.pivot(index='mass', columns='stem', values=['inty','lx1_bad_inty'])

In [ ]:
# TODO collape_join_adjecent_clusters


# MS2 
----

In [ ]:
options["MSMSresolution"]
options["selectionWindow"]

#tolerance = TypeTolerance("Da", scan.options["selectionWindow"])
#window = scan.options["selectionWindow"] / 2

In [ ]:
df = spectra_dfs[1] # first file, already teim range and mass range filtered
ms2_peaks = pd.concat((df.loc[~df.precursor_id.isna()] for df in spectra_dfs))

ms2_peaks[ms2_peaks.scan_id == 'controllerType=0 controllerNumber=1 scan=41']  
*** NOTE masses are not exactly the same as in lx1, they are wrong base om raw ***  
see xcel file on desktop  

*** lx1 does the precursor binning all files, at the same time, it is not like with the ms1s ***

In [ ]:
def bin_linear_alignment_for_ms2(masses, telerance_da):
    # TODO assert masses are ordered
    up_to = None
    for _, mass in masses.iteritems():
        if up_to is None:
            #up_to = mass + tolerance.getTinDA(mass) this is how its done in some places, but reuslt are not identical to below
            up_to = mass + telerance_da
        if mass <= up_to:
            yield up_to
        else:
            up_to = mass + telerance_da
            yield up_to
    

In [ ]:
def grouped_precursors_df(ms2_peaks):
    ms2_peaks.sort_values(['precursor_mz','mz'], inplace=True)
    precursors_df = ms2_peaks[["stem", 'scan_id', 'precursor_mz']].drop_duplicates()# similar to unqie but return a series instead of an array
    # using "stem", 'scan_id' to replicate the numebr of instances for the averaging later
    
    bins1 = list(bin_linear_alignment_for_ms2(precursors_df.precursor_mz, options["selectionWindow"]))
    bins2 = list(bin_linear_alignment_for_ms2(precursors_df.groupby(bins1)['precursor_mz'].transform('mean'), options["selectionWindow"]))
    bins3 = list(bin_linear_alignment_for_ms2(precursors_df.groupby(bins2)['precursor_mz'].transform('mean'), options["selectionWindow"]))

    precursors_df['prec_bin'] = bins3

# no fadi filter for precursors  
# after groupinbg the precursors, the groups are split on the "sample" ie the file,  
# the each split of the group is merged in the __def mergeListsMsms__ method that uses linear alignment  
# thats why... prec_ms2_peaks = ms2_peaks[(ms2_peaks.precursor_mz == t) & (ms2_peaks.stem == '190321_Serum_Lipidextract_368723_01')]
    return precursors_df

In [ ]:
precursors_df = grouped_precursors_df(ms2_peaks)
precursors_bins = precursors_df.set_index('precursor_mz')['prec_bin'].to_dict()

In [ ]:
precursors_bins

in def linearAlignment the var clusterToMerge, it only takes the "sample" if its not in the cluster, so no duplicate, and only the first "sample"

for clutering ms2 precursors, it does the binning with linearAlignment... see above, its not the unique precusrsor, its each precursor in each file,  but it does the binning on both at the same time, in contrast to how its done for ms1 binning, where first the binning is done for one file and then for the other.

then it does the merge it on a per file (AKA "sample")  basis, merge means doing a linera alignmebt on all the peaks in the bin per sample


In [ ]:
ms2_peaks['prec_bin'] = ms2_peaks.precursor_mz.map(precursors_bins)


In [ ]:
# this one uses delta res for clustering
def bin_mkSurveyLinear_for_ms2(masses, options): # copied from above
    # TODO assert masses are ordered
    minmass = masses.iloc[0]
    up_to = None
    for _, mass in masses.iteritems():
        if up_to is None:
            #up_to = mass + tolerance.getTinDA(mass) this is how its done in some places, but reuslt are not identical to below
            deltatol = options["MSMSresolution"].tolerance + (mass - minmass) * options['MSMSresolutionDelta']
            up_to = mass + (mass / deltatol)
        if mass <= up_to:
            yield up_to
        else:
            deltatol = options["MSMSresolution"].tolerance + (mass - minmass) * options['MSMSresolutionDelta']
            up_to = mass + (mass / deltatol)
            yield up_to

In [ ]:
def ms2_peaks_group_generator(grouped_prec, options):

    for idx, prec_ms2_peaks in  grouped_prec:
        #intensityWeightedAvg
        prec_ms2_peaks.sort_values('mz', inplace=True)
        bins1 = list(bin_mkSurveyLinear_for_ms2(prec_ms2_peaks.mz, options))
        bins1_weighted_average = (prec_ms2_peaks.mz * prec_ms2_peaks.inty).groupby(bins1).transform('sum') / prec_ms2_peaks.inty.groupby(bins1).transform('sum')
        bins2 = list(bin_mkSurveyLinear_for_ms2(bins1_weighted_average, options))
        bins2_weighted_average = (prec_ms2_peaks.mz * prec_ms2_peaks.inty).groupby(bins2).transform('sum') / prec_ms2_peaks.inty.groupby(bins2).transform('sum')
        bins3 = list(bin_mkSurveyLinear_for_ms2(bins2_weighted_average, options))
        weighted_mass = (prec_ms2_peaks.mz * prec_ms2_peaks.inty).groupby(bins3).transform('sum') / prec_ms2_peaks.inty.groupby(bins3).transform('sum')

        prec_ms2_peaks['bins'] = bins3
        prec_ms2_peaks['weighted_mass'] = weighted_mass

        fadi_denominator = prec_ms2_peaks.scan_id.unique().shape[0]
        ff_mask = prec_ms2_peaks.groupby("bins")['bins'].transform('count') / fadi_denominator>= options['MSMSfilter']
        mof_mask = prec_ms2_peaks.groupby("bins")['bins'].transform('count') / fadi_denominator>= options['MSMSminOccupation']

        tf_mask = prec_ms2_peaks.inty > options["MSMSthreshold"]
        
        # it uses merge sum intensity for getting the averrage intensity...
        agg_prec_ms2_peaks = prec_ms2_peaks[ff_mask & tf_mask & mof_mask].groupby('bins').agg({"weighted_mass":['mean', 'count'],"inty":"mean"})
        agg_prec_ms2_peaks['precursor_mz'] = prec_ms2_peaks.precursor_mz.mean().round(6) 
        # there is minor differrence in mean between different files, and the same precursor bin, to avoid it we round
        
        agg_prec_ms2_peaks['stem'] = idx[0]

        agg_prec_ms2_peaks.columns = ["_".join(col).strip() for col in agg_prec_ms2_peaks.columns.values]
        names = {'weighted_mass_mean':'mz',
         'weighted_mass_count':'count', 
         "inty_mean":'inty',
         'precursor_mz_':'precursor_mz',
         'stem_':'stem'}
        agg_prec_ms2_peaks.rename(columns = names, inplace=True)

        yield agg_prec_ms2_peaks



In [ ]:
ms2_peaks.prec_bin.unique().size

In [ ]:
grouped_prec = ms2_peaks.groupby(['stem','prec_bin'])
ms2_agg_peaks = pd.concat(ms2_peaks_group_generator(grouped_prec, options))

In [ ]:
# collape_join_adjecent_clusters_msms(cluster)

In [ ]:
# associate ms2 to ms1 scans... 
ms1_masses = ms1_agg_peaks.mass.drop_duplicates()
precur_masses = ms2_agg_peaks.precursor_mz.drop_duplicates()
tol = options["selectionWindow"] / 2
tmp = pd.merge_asof(ms1_masses, precur_masses.astype(ms1_masses.dtype), left_on='mass', right_on='precursor_mz', direction ='nearest' , tolerance=tol)

In [ ]:
ms1_agg_peaks

# build the masterscan

In [37]:
from lx.spectraContainer import MasterScan
from LX2_masterscan import se_factory, ms2entry_factory

[12:27:33] {comtypes.client._code_cache:95} INFO - Imported existing <module 'comtypes.gen' from 'c:\\Users\\mirandaa\\Anaconda3\\envs\\lx128_p3_Dev\\lib\\site-packages\\comtypes\\gen\\__init__.py'>
[12:27:33] {comtypes.client._code_cache:72} INFO - Using writeable comtypes cache directory: 'c:\Users\mirandaa\Anaconda3\envs\lx128_p3_Dev\lib\site-packages\comtypes\gen'


In [ ]:
def mass_inty_generator_ms1_agg(ms1_agg_peaks):
    for mass, gdf in ms1_agg_peaks.groupby('mass'):
        dictIntensity = gdf.set_index('stem')['lx1_bad_inty'].to_dict()
        dictIntensity_lx2 = gdf.set_index('stem')['inty'].to_dict()
        dictIntensity.update({ f'{k}_lx2':v for k,v in dictIntensity_lx2.items()})
        yield (mass, dictIntensity, 1)

In [ ]:
samples = samples[:2]

In [ ]:
samples.extend([f'{k}_lx2' for k in samples]) #because we add both results

In [ ]:
samples

In [ ]:
listSurveyEntry = [
        se_factory(msmass, dictIntensity, samples, polarity)
        for msmass, dictIntensity, polarity in mass_inty_generator_ms1_agg(
            ms1_agg_peaks
        )
    ]

In [ ]:
def MSMSEntry_list_generator(gdf):
    for mz,  precur_df in gdf.groupby('mz'):
        dictIntensity = precur_df.set_index('stem')['inty'].to_dict()
        dictIntensity.update({ f'{k}_lx2':v for k,v in dictIntensity.items()}) #TODO actually get other values
        yield (mz, dictIntensity, samples, 1)

In [ ]:
def MS2_dict_generator(ms2_agg_peaks):
    for precursor_mz, gdf in ms2_agg_peaks.groupby('precursor_mz'):
        MSMSEntry_list = [ms2entry_factory(*args) for args in MSMSEntry_list_generator(gdf)]
        yield (precursor_mz, MSMSEntry_list)

In [ ]:
MS2_dict = dict(MS2_dict_generator(ms2_agg_peaks))

In [ ]:
MS2_dict_keys =pd.Series(list(MS2_dict.keys()), name = 'MS2_precurs')

In [ ]:
MS2_dict_keys.sort_values(inplace=True)

In [ ]:
MS1_precurmass =pd.Series([se.precurmass for se in listSurveyEntry], name = 'MS1_precurmass')

In [ ]:
MS1_precurmass.sort_values(inplace=True)

In [ ]:
# associate ms2 to ms1 scans
tol = options["selectionWindow"] / 2
precur_map_df = pd.merge_asof(MS1_precurmass, MS2_dict_keys, left_on='MS1_precurmass' , right_on='MS2_precurs',direction ='nearest' , tolerance=tol)
# tmp = pd.merge_asof(ms1_masses, ms1_masses.dtype, left_on='mass', right_on='precursor_mz', direction ='nearest' , tolerance=tol)

In [ ]:
precur_dict = precur_map_df.set_index(MS1_precurmass)['MS2_precurs'].to_dict()

In [ ]:
len(precur_dict)

In [ ]:
# ms1_prec_dict = ms1_agg_peaks.set_index('mass')['precursor_mz'].to_dict()
for se in listSurveyEntry:
    precursor = precur_dict[se.precurmass]
    print(len(MS2_dict.get(precursor,[])))
    

In [36]:

# add data to masterscan
scan = MasterScan(options)
scan.listSurveyEntry = listSurveyEntry
scan.listSurveyEntry[0].massWindow = 0.01  # to avoid bug
scan.sampleOccThr["MS"] = [(0.0, [])]  # to avoid bug at def checkOccupation
scan.sampleOccThr["MSMS"] = [(0.0, [])]

# for printing we need
# samples.extend([f'{k}_lx2' for k in samples])
scan.listSamples = samples


NameError: name 'MasterScan' is not defined

In [ ]:
import pickle
with open('tmp_lx1_and_lx2.sc','wb') as f: pickle.dump(scan, f)

# recalibrate
import pickle

In [ ]:
import pickle
with open('ms1_dfs.pkl', 'rb') as f: ms1_dfs = pickle.load(f)
with open('options.pkl', 'rb') as f: options = pickle.load(f)

In [ ]:
#recalibrate
def recalibrate_mzs(mzs, cals):
    # from lx2_masterscan
    # lx1 takes all that are within tolerance and then uses highest intensity
    if not cals or mzs.empty:
        return mzs
    cal_matchs = [mzs.loc[mzs.sub(cal).abs().idxmin()] for cal in cals]

    cal_vals = [cal - cal_match for cal, cal_match in zip(cals, cal_matchs)]
    # prefilter
    if not any((v < 0.1 for v in cal_vals)):
        return mzs
    # find near tolerance
    cutoff = mzs.diff(-1).quantile(0.1)
    is_near = [v < cutoff for v in cal_vals]
    if not any(is_near):
        log.info("no valid calibration masses found")
        return mzs

    cal_matchs = [e for e, v in zip(cal_matchs, is_near) if v]
    cal_vals = [e for e, v in zip(cal_vals, is_near) if v]
    log.debug("recalibration info: {'\n'.join(zip(cal_matchs,cal_vals ))}")

    return mzs + np.interp(mzs, cal_matchs, cal_vals)

In [ ]:
res = []
for cal_mass in options['MScalibration']:
    tol = cal_mass / options["MSresolution"].tolerance
    #find close enough most intense
    reference_mass = ms1_df[ms1_df.mz.between(cal_mass-tol , cal_mass+tol)].sort_values('inty', ascending=False).mz.iat[0]
    change_val = cal_mass - reference_mass
    res.append((reference_mass, change_val))

cal_matchs, cal_vals = zip(*res)

# perf testing

In [2]:
import pickle

In [24]:
# with open('stitched.sc','wb') as f: pickle.dump(stitched, 

In [3]:
with open('stitched.sc','rb') as f: stitched  = pickle.load(f) 

In [4]:
stitched.dump('stitched-dump.csv')

Dumping Master Scan content
containing:
8488 MS entries and
0 MS/MS entries.


True

In [34]:
t = stitched.listSurveyEntry[0]
t.dictIntensity

{'Extraction_Blank_1_Ult-s': 339.1315002441406,
 'Extraction_Blank_2_Ult-s': 0,
 'Ultimate_1-s': 0,
 'Ultimate_2-s': 339.1653137207031,
 'Ult_05NIST_1-s': 274.6545104980469,
 'Ult_05NIST_2-s': 267.81512451171875,
 'Ult_10NIST_1-s': 0,
 'Ult_10NIST_2-s': 0,
 'Ult_1NIST_1-s': 261.4827575683594,
 'Ult_1NIST_2-s': 318.67169189453125,
 'Ult_5NIST_1-s': 0,
 'Ult_5NIST_2-s': 282.861083984375,
 'Extraction_Blank_1_Ult-s_lx2': 339.1315002441406,
 'Extraction_Blank_2_Ult-s_lx2': 0,
 'Ultimate_1-s_lx2': 0,
 'Ultimate_2-s_lx2': 339.1653137207031,
 'Ult_05NIST_1-s_lx2': 274.6545104980469,
 'Ult_05NIST_2-s_lx2': 267.81512451171875,
 'Ult_10NIST_1-s_lx2': 0,
 'Ult_10NIST_2-s_lx2': 0,
 'Ult_1NIST_1-s_lx2': 261.4827575683594,
 'Ult_1NIST_2-s_lx2': 318.67169189453125,
 'Ult_5NIST_1-s_lx2': 0,
 'Ult_5NIST_2-s_lx2': 282.861083984375}